In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tkinter as tk
from tkinter import messagebox

# Step 1: Load the dataset
file_path = 'collegedataset.csv'  # Update with the correct path if needed
data = pd.read_csv(file_path)

# Step 2: Feature engineering and target preparation
features = [
    'Percent of freshmen submitting SAT scores',
    'SAT Math 25th percentile score',
    'SAT Math 75th percentile score'
]

target = 'Admissions total'

# Convert target to binary (admitted or not)
threshold = data[target].median()
data['Admission_Status'] = (data[target] >= threshold).astype(int)
target = 'Admission_Status'

# Feature engineering
data['SAT_Math_Spread'] = data['SAT Math 75th percentile score'] - data['SAT Math 25th percentile score']
features.append('SAT_Math_Spread')

# Define X (features) and y (target)
X = data[features]
y = data[target]

# Handle missing data
X = X.fillna(X.mean())
y = y.fillna(y.mode()[0])

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Handle imbalanced data with SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Step 3: Train models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)

# Train Logistic Regression model
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train, y_train)

# Step 4: Test the models and select the best one based on accuracy
rf_y_pred = rf_model.predict(X_test)
logreg_y_pred = logreg_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_y_pred)
logreg_accuracy = accuracy_score(y_test, logreg_y_pred)

best_model = rf_model if rf_accuracy > logreg_accuracy else logreg_model

# Step 5: Define the function to make predictions
def predict_admission(sat_score_1, sat_score_2, sat_score_3):
    # Prepare input data
    sample_input = pd.DataFrame([[sat_score_1, sat_score_2, sat_score_3, sat_score_2 - sat_score_1]], columns=features)
    sample_input = scaler.transform(sample_input)  # Scale the input to match training data

    # Predict using the best model
    admission_prediction = best_model.predict(sample_input)

    return "Admitted" if admission_prediction[0] == 1 else "Not Admitted"

# Step 6: Create the GUI interface
def on_submit():
    try:
        # Get user input
        sat_score_1 = float(entry_sat_score_1.get())
        sat_score_2 = float(entry_sat_score_2.get())
        sat_score_3 = float(entry_sat_score_3.get())
        
        # Get prediction
        result = predict_admission(sat_score_1, sat_score_2, sat_score_3)

        # Show the result
        messagebox.showinfo("Prediction Result", f"Admission Status: {result}")
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numbers for all SAT scores.")

# Create the main window
root = tk.Tk()
root.title("College Admission Prediction")

# Create and pack widgets
label_sat_score_1 = tk.Label(root, text="SAT Score 1 (Percent of freshmen submitting SAT scores):")
label_sat_score_1.pack()

entry_sat_score_1 = tk.Entry(root)
entry_sat_score_1.pack()

label_sat_score_2 = tk.Label(root, text="SAT Math 25th percentile score:")
label_sat_score_2.pack()

entry_sat_score_2 = tk.Entry(root)
entry_sat_score_2.pack()

label_sat_score_3 = tk.Label(root, text="SAT Math 75th percentile score:")
label_sat_score_3.pack()

entry_sat_score_3 = tk.Entry(root)
entry_sat_score_3.pack()

# Submit button
submit_button = tk.Button(root, text="Predict Admission", command=on_submit)
submit_button.pack()

# Run the main loop
root.mainloop()
